## Children Household Count Per County File Processing

In this file, we will be making a dataframe that includes the number of children in all of the households in each county of the US.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import geopandas as gpd

### Importing county level shape file

This shapefile of US counties was also found online. This file provides the polygon geometries of each county in the US; this will allow us to make visualizations of the US with breakdowns by county. In this case, we are making this processing file to allow us make a map of US with children count by county.

In [3]:
df_counties = gpd.read_file('/hpc/group/codeplus22-vis/celine_data/source_files/county_shapefiles/counties.shp')

df_counties['county_fips'] = df_counties['STATEFP'] + df_counties['COUNTYFP']
df_counties

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,geometry,county_fips
0,20,161,00485044,0500000US20161,20161,Riley,Riley County,KS,Kansas,06,1579077672,32047392,"POLYGON ((-96.96095 39.28670, -96.96106 39.288...",20161
1,19,159,00465268,0500000US19159,19159,Ringgold,Ringgold County,IA,Iowa,06,1386932347,8723135,"POLYGON ((-94.47167 40.81255, -94.47166 40.819...",19159
2,30,009,01720111,0500000US30009,30009,Carbon,Carbon County,MT,Montana,06,5303728455,35213028,"POLYGON ((-109.79867 45.16734, -109.68779 45.1...",30009
3,16,007,00395090,0500000US16007,16007,Bear Lake,Bear Lake County,ID,Idaho,06,2527123155,191364281,"POLYGON ((-111.63452 42.57034, -111.63010 42.5...",16007
4,55,011,01581065,0500000US55011,55011,Buffalo,Buffalo County,WI,Wisconsin,06,1750290818,87549529,"POLYGON ((-92.08384 44.41200, -92.08310 44.414...",55011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3229,53,003,01533502,0500000US53003,53003,Asotin,Asotin County,WA,Washington,06,1647427905,11291731,"POLYGON ((-117.47999 46.12199, -117.41948 46.1...",53003
3230,13,043,00342852,0500000US13043,13043,Candler,Candler County,GA,Georgia,06,629520841,15018189,"POLYGON ((-82.25457 32.35150, -82.25276 32.353...",13043
3231,48,451,01384011,0500000US48451,48451,Tom Green,Tom Green County,TX,Texas,06,3941965409,48077315,"POLYGON ((-101.26763 31.55646, -101.25039 31.5...",48451
3232,39,089,01074057,0500000US39089,39089,Licking,Licking County,OH,Ohio,06,1767478831,12761090,"POLYGON ((-82.78181 39.94698, -82.78126 39.955...",39089


### Importing infousa data

This preprocessed infousa household data contains the children counts of households as well as the zipcode and county fips of each household. This file also includes the transformed latitude and longitude coordinates.

In this dataset, the county_fips column represents each individual county; this is the column we will be grouping the children count by. 

In [ ]:
df_hh = pd.read_parquet('/hpc/group/codeplus22-vis/celine_data/zip_00_99.parquet')
df_hh

In [ ]:
df_hh = df_hh[['county_fips', 'state', 'child_num', 'has_child', 'age_code']]
df_hh

### Groupby

Here, we are taking the infousa preprocessed file and calculating the number of children per county. We are doing so using the ```groupby``` function, which groups the county_fips together by summing the corresponding ```child_num``` values. The resulting dataframe is giving the calculated number of children per county_fips.

In [ ]:
df_child_count = df_hh.groupby('county_fips')['child_num'].sum().reset_index()
df_child_count

### Merging child number counts with county geometries

Now that we have the child_num calculated for each ```county_fips```, we are merging the county geometry shapefile with the child count dataframe on the ```county_fips``` column. Afterwards, we will select on the columns we want to keep in this dataframe, before exporting this as a parquet file to be used in future visualizations.

In [ ]:
df_final = df_child_count.merge(df_counties, on = ['county_fips'], how = 'left')

df_final

In [ ]:
df_final = df_final[['STATEFP', 'NAME', 'county_fips','child_num', 'geometry']]
df_final

In [ ]:
df_final.rename(columns = {'STATEFP': 'state', 'NAME':'county'}, inplace = True)
df_final

### Converting GeoDataFrame

In order to export this dataframe as a shapefile, the pandas dataframe must be converted to a GeoDataFrame using the ```gpd.GeoDataFrame()``` function. 

In [ ]:
gpd_df = gpd.GeoDataFrame(df_final)
gpd_df

In [ ]:
gpd_df.to_file('/hpc/group/codeplus22-vis/celine_data/children_count_by_county.shp')

In [3]:
df =gpd.read_file('/hpc/group/codeplus22-vis/celine_data/children_count_by_county.shp')
df

DriverError: /hpc/group/codeplus22-vis/celine_data/children_count_by_county.shp: No such file or directory